## import necessary csv

In [100]:
import pandas as pd
import numpy as np

In [101]:
events_italy = pd.read_csv("srcd_data/events_Italy.csv")
events_italy.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,tagsList,pos_orig_y,pos_orig_x,pos_dest_y,pos_dest_x
0,8,Simple pass,[{'id': 1801}],8327,"[{'y': 52, 'x': 49}, {'y': 44, 'x': 43}]",2575959,Pass,3158,1H,2.530536,85.0,180423957,[1801],52,49,44,43
1,8,Simple pass,[{'id': 1801}],20438,"[{'y': 44, 'x': 43}, {'y': 17, 'x': 36}]",2575959,Pass,3158,1H,3.768418,85.0,180423958,[1801],44,43,17,36
2,7,Touch,[],8306,"[{'y': 17, 'x': 36}, {'y': 56, 'x': 78}]",2575959,Others on the ball,3158,1H,4.868265,72.0,180423959,NaN,17,36,56,78
3,1,Ground attacking duel,"[{'id': 504}, {'id': 703}, {'id': 1801}]",8306,"[{'y': 56, 'x': 78}, {'y': 15, 'x': 64}]",2575959,Duel,3158,1H,8.114676,11.0,180423960,"[504, 703, 1801]",56,78,15,64
4,1,Ground attacking duel,"[{'id': 503}, {'id': 703}, {'id': 1801}]",8306,"[{'y': 15, 'x': 64}, {'y': 15, 'x': 72}]",2575959,Duel,3158,1H,8.647892,11.0,180423961,"[503, 703, 1801]",15,64,15,72


## filter necessary columns

In [250]:
italy_fouls = events_italy[events_italy["eventName"] == "Foul"]
italy_fouls = italy_fouls[['eventId', 'subEventName', 'matchId', 'teamId', 'matchPeriod', 'eventSec']]
italy_fouls.head()

,eventId,subEventName,matchId,teamId,matchPeriod,eventSec
25,2,Foul,2575959,3172,1H,33.869510
48,2,Foul,2575959,3172,1H,123.664460
151,2,Foul,2575959,3172,1H,463.985339
179,2,Foul,2575959,3158,1H,536.411890
221,2,Foul,2575959,3158,1H,644.020804


In [251]:
## adjust all instances to pure minute 
italy_fouls.loc[italy_fouls["matchPeriod"] == "2H", "eventSec"] += (45 * 60)

In [252]:
italy_fouls = italy_fouls[['eventId', 'subEventName', 'matchId', 'teamId', 'eventSec']]
italy_fouls["eventMin"] = round(italy_fouls["eventSec"] / 60, 0)
italy_fouls.head()

,eventId,subEventName,matchId,teamId,eventSec,eventMin
25,2,Foul,2575959,3172,33.869510,1.0
48,2,Foul,2575959,3172,123.664460,2.0
151,2,Foul,2575959,3172,463.985339,8.0
179,2,Foul,2575959,3158,536.411890,9.0
221,2,Foul,2575959,3158,644.020804,11.0


## group fouls by game

In [253]:
fouls_ind_by_game = dict((italy_fouls.groupby("matchId").apply(lambda x: list(dict(x["eventMin"]).values()), include_groups=False)))

In [254]:
dInfo = {"matchId": ["filler"], "fouls": [["fller"]]}
italy_fbm = pd.DataFrame(dInfo) 
italy_fbm = italy_fbm.drop(index=0) 
italy_fbm["matchId"] = fouls_ind_by_game.keys()
italy_fbm["fouls"] = fouls_ind_by_game.values()


In [255]:
fouls_per_game = [] 
late_game_fouls_by_game = [] 
for row in italy_fbm.iterrows():
    game_fouls = len(row[1]["fouls"])
    lg_fouls = len(list(filter(lambda x: x >= 60, row[1]["fouls"])))

    late_game_fouls_by_game.append(lg_fouls)
    fouls_per_game.append(game_fouls)

In [258]:
italy_fbm["total fouls"] = fouls_per_game
italy_fbm["late game fouls"] = late_game_fouls_by_game
italy_fbm["late game foul density"] =  italy_fbm["late game fouls"] / italy_fbm["total fouls"]

In [259]:
italy_fbm

,matchId,fouls,total fouls,late game fouls,late game foul density
0,2575959,"[1.0, 2.0, 8.0, 9.0, 11.0, 17.0, 20.0, 22.0, 2...",29,10,0.344828
1,2575960,"[2.0, 13.0, 18.0, 19.0, 20.0, 29.0, 31.0, 39.0...",28,11,0.392857
2,2575961,"[2.0, 6.0, 8.0, 8.0, 11.0, 11.0, 13.0, 19.0, 2...",24,6,0.250000
3,2575962,"[12.0, 23.0, 25.0, 27.0, 35.0, 37.0, 40.0, 45....",17,8,0.470588
4,2575963,"[4.0, 6.0, 10.0, 13.0, 15.0, 16.0, 29.0, 45.0,...",20,9,0.450000
...,...,...,...,...,...
375,2576334,"[2.0, 3.0, 5.0, 5.0, 15.0, 16.0, 20.0, 21.0, 2...",24,6,0.250000
376,2576335,"[8.0, 12.0, 19.0, 20.0, 22.0, 23.0, 25.0, 33.0...",28,13,0.464286
377,2576336,"[3.0, 10.0, 14.0, 16.0, 21.0, 23.0, 25.0, 28.0...",24,7,0.291667
378,2576337,"[0.0, 2.0, 7.0, 7.0, 11.0, 12.0, 26.0, 28.0, 2...",23,8,0.347826


## relevant df's 

In [261]:
print(italy_fouls)

        eventId subEventName  matchId  teamId     eventSec  eventMin
25            2         Foul  2575959    3172    33.869510       1.0
48            2         Foul  2575959    3172   123.664460       2.0
151           2         Foul  2575959    3172   463.985339       8.0
179           2         Foul  2575959    3158   536.411890       9.0
221           2         Foul  2575959    3158   644.020804      11.0
...         ...          ...      ...     ...          ...       ...
646955        2         Foul  2576338    3185  4156.018348      69.0
647004        2         Foul  2576338    3193  4269.079094      71.0
647145        2         Foul  2576338    3185  4681.807621      78.0
647211        2         Foul  2576338    3185  4955.719625      83.0
647299        2         Foul  2576338    3185  5338.693956      89.0

[9992 rows x 6 columns]


In [262]:
print(italy_fbm)

     matchId                                              fouls  total fouls  \
0    2575959  [1.0, 2.0, 8.0, 9.0, 11.0, 17.0, 20.0, 22.0, 2...           29   
1    2575960  [2.0, 13.0, 18.0, 19.0, 20.0, 29.0, 31.0, 39.0...           28   
2    2575961  [2.0, 6.0, 8.0, 8.0, 11.0, 11.0, 13.0, 19.0, 2...           24   
3    2575962  [12.0, 23.0, 25.0, 27.0, 35.0, 37.0, 40.0, 45....           17   
4    2575963  [4.0, 6.0, 10.0, 13.0, 15.0, 16.0, 29.0, 45.0,...           20   
..       ...                                                ...          ...   
375  2576334  [2.0, 3.0, 5.0, 5.0, 15.0, 16.0, 20.0, 21.0, 2...           24   
376  2576335  [8.0, 12.0, 19.0, 20.0, 22.0, 23.0, 25.0, 33.0...           28   
377  2576336  [3.0, 10.0, 14.0, 16.0, 21.0, 23.0, 25.0, 28.0...           24   
378  2576337  [0.0, 2.0, 7.0, 7.0, 11.0, 12.0, 26.0, 28.0, 2...           23   
379  2576338  [1.0, 2.0, 3.0, 13.0, 20.0, 22.0, 33.0, 35.0, ...           19   

     late game fouls  late game foul de

In [263]:
italy_fouls.to_csv("prepped_data/italy_fouls.csv")

In [264]:
italy_fbm.to_csv("prepped_data/italy_fbm.csv")